### The `Selection` Widgets

+ Dropdown
+ RadioButtons
+ ToggleButtons
+ Select
+ SelectMultiple

These widgets can be used to choose between multiple alternatives. The `SelectMultiple` widget allows multiple selections, whereas `Dropdown`, `RadioButtons`, `ToggleButtons`, and `Select` only allow one selection.

In [1]:
{-# LANGUAGE OverloadedStrings #-}
import IHaskell.Display.Widgets

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:

In [2]:
-- Allows single selection
tgbs <- mkToggleButtons

-- Allows multiple selections
msel <- mkSelectMultiple

In [3]:
setField msel Description "Functions to show (One or more)"
setField msel Options (OptionLabels ["sin", "cos"])

setField tgbs Description "Plot style"
setField tgbs Options (OptionLabels ["line", "point"])

The cell below requires `Chart` and `Chart-cairo` to be installed.

In [46]:
import Graphics.Rendering.Chart.Easy hiding (tan)
import Graphics.Rendering.Chart.Backend.Cairo
import qualified Data.ByteString as B
import Data.Text (pack, unpack)
import IHaskell.Display (base64)
import Control.Applicative ((<$>))

import Control.Monad (when, forM)
import Data.Maybe (fromJust)
import Data.List

dset :: [(String, [(Double, Double)])]
dset = [("sin", zmap sin r), ("cos", zmap cos r)]
  where zmap f xs = zip xs (map f xs)
        r = [0, 0.1 .. 6.3]

i <- mkImageWidget
setField i Width 500
setField i Height 500

-- Redraw the plot based on values from the widgets
refresh = do
  -- Read values from the widgets
  funs <- getField msel Indices
  sty <- getField tgbs Index

  let pts = fmap (genericIndex dset) funs
      opts = def { _fo_size = (500, 500) }

  toFile opts ".chart" $ do
    layout_title .= "Plotting: " ++ unwords (fmap fst pts)
    if sty == 0
      then mapM_ (\(s, ps) -> plot (line s [ps])) pts
      else mapM_ (\(s, ps) -> plot (points s ps)) pts

  img <- B.readFile ".chart"
  setField i B64Value (base64 img)
  
-- Add event handlers to make widgets work
setField msel SelectionHandler refresh
setField tgbs SelectionHandler refresh

-- Trigger event to show empty grid initially
triggerSelection msel

In [41]:
-- Display the widgets
msel
tgbs
i

The `Dropdown`, `RadioButtons` and `Select` widgets behave just like the `ToggleButtons` widget. They have the same properties, and the same functionality.